<a href="https://colab.research.google.com/github/ThamiGG/restaindian/blob/main/RestauranteIndiano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DESAFIO BI - ALURA**

Origem dos dados disponibilizado pela Alura: Kaggle (Zomato)

1. Instalação e importações

In [ ]:
!pip install requests

import pandas as pd
import numpy as np
import json
import requests

from google.colab import drive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


2. Conectar ao drive

In [ ]:
drive.mount('/content/gdrive')
# diretório dos arquivos
root_dir = '/content/gdrive/MyDrive/CienciaDeDados/DesafioAlura/RestauranteIndiano'
# ir ao diretório
%cd $root_dir


Mounted at /content/gdrive
/content/gdrive/MyDrive/CienciaDeDados/DesafioAlura/RestauranteIndiano


3. Leitura e limpeza dos dados

In [ ]:
json_file_path = "/content/gdrive/MyDrive/CienciaDeDados/DesafioAlura/RestauranteIndiano/"

arq = 5 #quantidade de arquivos
nome = 'file'

for i in range(arq):
  data = pd.read_json(json_file_path + nome + str(i+1) + '.json')
  if i != 0:
    rest0 = pd.DataFrame(data['restaurants'])
    rest = pd.concat([rest, rest0])
  else:
    rest = pd.DataFrame(data['restaurants'])
a = rest.explode(column='restaurants', ignore_index=True) #tirar os dicionarios das []
b = pd.json_normalize(data=a['restaurants']) #distribuir os dados do dicionario em várias colunas
c = b.drop_duplicates(subset='restaurant.id') #tira ids duplicados e mantem o primeiro
d = c.drop(['restaurant.url','restaurant.photos_url','restaurant.apikey', #exclui colunas
            'restaurant.zomato_events','restaurant.featured_image',
            'restaurant.location.zipcode','restaurant.deeplink','restaurant.menu_url',
            'restaurant.book_url','restaurant.switch_to_order_menu','restaurant.offers',
            'restaurant.has_table_booking','restaurant.location.locality_verbose',
            'restaurant.location.locality','restaurant.R.res_id',
            'restaurant.thumb','restaurant.establishment_types','restaurant.events_url',
            'restaurant.order_deeplink','restaurant.order_url'], axis=1) 
e = d.rename(columns={ #renomeia colunas
    "restaurant.has_online_delivery":"entrega online",
    "restaurant.price_range":"faixa de preço da comida",
    "restaurant.user_rating.rating_text":"classificação textual",
    "restaurant.user_rating.rating_color":"classificação em cores",
    "restaurant.user_rating.votes":"numero de votos",
    "restaurant.user_rating.aggregate_rating":"classificação de 0 a 5",
    "restaurant.id":"id",
    "restaurant.name":"nome",
    "restaurant.cuisines":"culinária",
    "restaurant.is_delivering_now":"entrega",
    "restaurant.average_cost_for_two":"custo medio para 2",
    "restaurant.location.latitude":"latitude",
    "restaurant.location.longitude":"longitude",
    "restaurant.location.address":"endereço",
    "restaurant.location.city":"cidade",
    "restaurant.location.country_id":"id pais",
    "restaurant.location.city_id":"id cidade",
    "restaurant.currency":"moeda"})
df = e.set_index('id') #determina o indice


Para analisar os tipos de culinária foi criado um arquivo a parte para não perder a chave primária, pois um id pode ter mais um tipo de culinária.

In [ ]:
f1 = df['culinária'].copy() #copia determinada coluna
f2 = df['classificação de 0 a 5'].copy()
g1 = f1.str.split(',',expand=True) #separa os dados em novas colunas
h1 = pd.melt(g1,value_vars=[0,1,2,3,4,5,6,7],value_name='culinaria',ignore_index=False) #as novas colunas se separam em linhas
h = h1.join(f2) #a classficação dada ao restaurante é adicionada
df2 = h.dropna() #remove linhas nulas
df2.to_csv('/content/gdrive/MyDrive/CienciaDeDados/DesafioAlura/RestauranteIndiano/culinaria.csv') #salva o arquivo

Os dados de custo médio da refeição para duas pessoas vieram na moeda local, portanto cada valor foi convertido para reais.

In [ ]:
#a API com a conversão de moeda é requerida
req = requests.get('http://economia.awesomeapi.com.br/json/last/USD-BRL,INR-BRL,AUD-BRL,CAD-BRL,BRL-IDR,NZD-BRL,BRL-PHP,BRL-QAR,SGD-BRL,ZAR-BRL,BRL-LKR,TRY-BRL,AED-BRL,GBP-BRL')
req = req.json()

#atribui o valor de conversão 
cot_india = float(req['INRBRL']['bid'])
cot_austl = float(req['AUDBRL']['bid'])
cot_canad = float(req['CADBRL']['bid'])
cot_indon = float(req['BRLIDR']['bid'])
cot_filip = float(req['BRLPHP']['bid'])
cot_qatar = float(req['BRLQAR']['bid'])
cot_singp = float(req['SGDBRL']['bid'])
cot_afris = float(req['ZARBRL']['bid'])
cot_srlak = float(req['BRLLKR']['bid'])
cot_turqi = float(req['TRYBRL']['bid'])
cot_arabe = float(req['AEDBRL']['bid'])
cot_rundo = float(req['GBPBRL']['bid'])
cot_eundo = float(req['USDBRL']['bid'])
cot_zelan = float(req['NZDBRL']['bid'])

#de acordo com o id do pais se faz a conversão 
for index,row in df.iterrows():
  if row['id pais']==1.0:
    df.loc[index,'custo medio para 2'] = row['custo medio para 2']*cot_india
  elif row['id pais']==14.0:
    df.loc[index,'custo medio para 2'] = row['custo medio para 2']*cot_austl
  elif row['id pais']==37.0:
    df.loc[index,'custo medio para 2'] = row['custo medio para 2']*cot_canad
  elif row['id pais']==94.0:
    df.loc[index,'custo medio para 2'] = row['custo medio para 2']/cot_indon
  elif row['id pais']==148.0:
    df.loc[index,'custo medio para 2'] = row['custo medio para 2']*cot_zelan
  elif row['id pais']==162.0:
    df.loc[index,'custo medio para 2'] = row['custo medio para 2']/cot_filip
  elif row['id pais']==166.0:
    df.loc[index,'custo medio para 2'] = row['custo medio para 2']/cot_qatar
  elif row['id pais']==184.0:
    df.loc[index,'custo medio para 2'] = row['custo medio para 2']*cot_singp
  elif row['id pais']==189.0:
    df.loc[index,'custo medio para 2'] = row['custo medio para 2']*cot_afris
  elif row['id pais']==191.0:
    df.loc[index,'custo medio para 2'] = row['custo medio para 2']/cot_srlak
  elif row['id pais']==208.0:
    df.loc[index,'custo medio para 2'] = row['custo medio para 2']*cot_turqi
  elif row['id pais']==214.0:
    df.loc[index,'custo medio para 2'] = row['custo medio para 2']*cot_arabe
  elif row['id pais']==215.0:
    df.loc[index,'custo medio para 2'] = row['custo medio para 2']*cot_rundo
  elif row['id pais']==216.0:
    df.loc[index,'custo medio para 2'] = row['custo medio para 2']*cot_eundo

id
308322        1.0
18037817      1.0
312345        1.0
307490        1.0
18241537      1.0
            ...  
17582627    216.0
17582625    216.0
17582668    216.0
17582498    216.0
17582499    216.0
Name: id pais, Length: 9578, dtype: float64 id
308322      1600.0
18037817    1500.0
312345       850.0
307490      1850.0
18241537    1600.0
             ...  
17582627       0.0
17582625      40.0
17582668      45.0
17582498      10.0
17582499       0.0
Name: custo medio para 2, Length: 9578, dtype: float64
DEPOIS
id
308322        1.0
18037817      1.0
312345        1.0
307490        1.0
18241537      1.0
            ...  
17582627    216.0
17582625    216.0
17582668    216.0
17582498    216.0
17582499    216.0
Name: id pais, Length: 9578, dtype: float64 id
308322      102.3680
18037817     95.9700
312345       54.3830
307490      118.3630
18241537    102.3680
              ...   
17582627      0.0000
17582625    211.5960
17582668    238.0455
17582498     52.8990
17582499      0.0000
Na

In [ ]:
df.to_csv('/content/gdrive/MyDrive/CienciaDeDados/DesafioAlura/RestauranteIndiano/restaurantes.csv') 